# studying the distribution of returns of NIFTY indices

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()
indexName <- "NIFTY 50 TR"

In [ ]:
trIndex <- data.frame(indices$NseTimeSeries() %>%
    filter(NAME == indexName) %>%
    select(TIME_STAMP, CLOSE))

trXts <- xts(trIndex$CLOSE, as.Date(trIndex$TIME_STAMP))
monthlyTrReturns <- monthlyReturn(trXts)

#ignore the first and last months
monthlyTrReturns <- monthlyTrReturns[-1]
monthlyTrReturns <- monthlyTrReturns[-nrow(monthlyTrReturns)]

monthlyTrReturns <- monthlyTrReturns * 100

In [ ]:
# statistical summary of returns
print(summary(monthlyTrReturns))

In [ ]:
firstDate <- first(index(monthlyTrReturns))
lastDate <- last(index(monthlyTrReturns))
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)

mRetDf <- data.frame(monthlyTrReturns)
names(mRetDf) <- c('R')

mRetDf$T <- index(monthlyTrReturns)

ggplot(mRetDf, aes(x=R)) +
    theme_economist() +
    geom_density() +
    labs(x='%', y='density', color='', title=sprintf("%s Density Plot", indexName), 
         subtitle=sprintf("monthly returns [%s:%s]", firstDate, lastDate)) +
    annotate("text", x=max(monthlyTrReturns), y=0, label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

### super-impose a normal distribution with the same mean and standard-deviation

what are these "fat-tails" that people keep talking about?

In [ ]:
avg <- mean(coredata(monthlyTrReturns))
stddev <- sd(coredata(monthlyTrReturns))

mRetDf$N1 <- rnorm(nrow(monthlyTrReturns), mean=avg, sd=stddev)
mRetDf$N2 <- rnorm(nrow(monthlyTrReturns), mean=avg, sd=stddev)
mRetDf$N3 <- rnorm(nrow(monthlyTrReturns), mean=avg, sd=stddev)
mRetDf$N4 <- rnorm(nrow(monthlyTrReturns), mean=avg, sd=stddev)
mRetDf$N5 <- rnorm(nrow(monthlyTrReturns), mean=avg, sd=stddev)

ggplot(mRetDf) +
    theme_economist() +
    geom_density(aes(x = R), color='blue') +
    geom_density(aes(x = N1), color='lightsalmon') +
    geom_density(aes(x = N2), color='lightsalmon') +
    geom_density(aes(x = N3), color='lightsalmon') +
    geom_density(aes(x = N4), color='lightsalmon') +
    geom_density(aes(x = N5), color='lightsalmon') +
    labs(x='%', y='density', color='', title=sprintf("%s Density Plot", indexName), 
         subtitle=sprintf("monthly returns [%s:%s]", firstDate, lastDate)) +
    annotate("text", x=max(monthlyTrReturns), y=0, label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)